In [1]:
import sys
sys.path.append('..')
import utils

# data input

In [5]:
from mxnet import ndarray as nd

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

num_inputs = 28*28
num_outputs = 10

num_hidden = 256
weight_scale = .01

w1 = nd.random_normal(shape=(num_inputs, num_hidden), scale=weight_scale)
b1 = nd.zeros(num_hidden)

w2 = nd.random_normal(shape=(num_hidden, num_outputs), scale=weight_scale)
b2 = nd.zeros(num_outputs)

params = [w1, b1, w2, b2]

for param in params:
    param.attach_grad()

# define model

In [6]:
# activation function
def relu(x):
    return nd.maximum(x, 0)

def net(x):
    x = x.reshape((-1, num_inputs))
    h1 = relu(nd.dot(x, w1) + b1)
    output = nd.dot(h1, w2) + b2
    return output

from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss() # softmax, corss entropy

# training

In [7]:
from mxnet import autograd as autograd

learning_rate = 0.5

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        utils.SGD(params, learning_rate / batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % \
          (epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

Epoch 0. Loss: 0.822970, Train acc 0.689320, Test acc 0.772135
Epoch 1. Loss: 0.493056, Train acc 0.817775, Test acc 0.844451
Epoch 2. Loss: 0.432237, Train acc 0.841112, Test acc 0.824920
Epoch 3. Loss: 0.394181, Train acc 0.853282, Test acc 0.860076
Epoch 4. Loss: 0.370871, Train acc 0.862614, Test acc 0.866286
